In [ ]:
import sys
!{sys.executable} -m pip install langchain huggingface_hub

!{sys.executable} -m  pip install bitsandbytes>=0.39.0
!{sys.executable} -m pip install --upgrade accelerate
!{sys.executable} -m pip install --upgrade sentence_transformers
!{sys.executable} -m  pip install --upgrade transformers
!{sys.executable} -m pip install --upgrade trl
!{sys.executable} -m pip install --upgrade peft
!{sys.executable} -m pip install --upgrade sqlite3
!{sys.executable} -m pip install --upgrade pytz
!{sys.executable} -m pip install --upgrade pypdf PyPDF2
!{sys.executable} -m pip install --upgrade langchain_experimental
!{sys.executable} -m pip install --upgrade langchain_community
!{sys.executable} -m pip install --upgrade langchain_openai
!{sys.executable} -m pip install --upgrade openai tiktoken chromadb
!pip install unstructured['pdf'] unstructured
!{sys.executable} -m pip install --upgrade textwrap
!pip install --upgrade --quiet  cohere
!pip install langchain-cohere
!pip install grandalf
!pip install wikipedia
!pip install faiss-cpu
!pip install faiss-gpu
!pip install langchainhub
!pip install -qU langchain-mistralai
!pip install "unstructured[html]"
!pip install bs4
!pip install rank_bm25
!pip install gradio

import gradio as gr
from huggingface_hub import InferenceClient

import os
import time
import sqlite3
import re
from pytz import timezone
import pytz
import datetime
import json
import textwrap
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
import torch
from operator import itemgetter
import os
import sys

import peft
import bitsandbytes
import accelerate
import trl
from accelerate import init_empty_weights
import torch
import pandas as pd

from transformers import AutoConfig, BitsAndBytesConfig
from accelerate.utils import BnbQuantizationConfig, load_and_quantize_model
from accelerate import load_checkpoint_and_dispatch
import accelerate, bitsandbytes

from langchain_mistralai import ChatMistralAI
import getpass
from google.colab import userdata


from langchain import hub
from langchain import PromptTemplate, HuggingFaceHub, LLMChain
from langchain_experimental.agents import create_csv_agent
from langchain.agents.agent_types import AgentType
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
from langchain_openai import ChatOpenAI
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from langchain_community.llms import HuggingFaceEndpoint
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain_community.embeddings import HuggingFaceInferenceAPIEmbeddings
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain.document_loaders import UnstructuredPDFLoader, PyMuPDFLoader
from langchain_community.document_loaders import UnstructuredHTMLLoader
from langchain_community.document_loaders import BSHTMLLoader
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import WebBaseLoader
from langchain.vectorstores import Chroma

from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain_cohere import CohereRerank

from langchain.chains import RetrievalQA
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableParallel, RunnableLambda
from langchain.llms import HuggingFaceHub
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain.callbacks.tracers import ConsoleCallbackHandler

from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import BaseTool, StructuredTool, tool
from langchain.tools.retriever import create_retriever_tool
from langchain.agents import create_tool_calling_agent
from langchain.agents import AgentExecutor
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain.agents import AgentType, initialize_agent
from langchain_core.messages import HumanMessage, ToolMessage, SystemMessage, AIMessage
from langchain_core.prompts import (
    ChatPromptTemplate,
    FewShotChatMessagePromptTemplate,
)

from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_core.tools import Tool
from langchain_experimental.utilities import PythonREPL
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from google.colab import userdata


Instantiate your llm - using Mistral in this example

In [ ]:
Mistral_TOKEN = userdata.get('mistralapi')
os.environ['MISTRAL_API_KEY'] = Mistral_TOKEN
llmMist = ChatMistralAI(model="mistral-large-latest", temperature =0)

Function to ingest Appwall security and system DBs in this example and return pandas dataframes

In [ ]:

def epochtotzone(ep,tzo= None):
  """
  Convert epoch values in db to DateTime
  """
  if not tzo:
    tzo = "Asia/Jerusalem"
  tzn=timezone(tzo)
  try:
      t=datetime.datetime.utcfromtimestamp(ep).replace(tzinfo=pytz.utc)
      x= t.astimezone(tzn)
      return t.strftime("%Y-%m-%dT%H:%M:%S")


def aw_to_pd(f,ftype='security',tzone=None):
  """
  Appwall security and system db to pandas dataframe
  """


  jsonlist=[]

  if 'security' in ftype:
      colls= ['DateTime','TargetPort','TargetIP','TransID','TunnelID','IsPassiveMode','Title','URI','Description']
  elif 'system' in ftype:
      colls=['DateTime','Title','Description']

  con = sqlite3.connect(f)
  con.text_factory = lambda b: b.decode(errors = 'ignore') # added to ignore utf-8 decode failure
  cursor = con.cursor()
  cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
  qu='SELECT * from %s'%'Events'
  df=pd.read_sql(qu,con)
  df=df[~df['DateTime'].isna()]
  df['DateTime']=df['DateTime'].apply(epochtotzone,args=(tzone,))
  df['DateTime'] = pd.to_datetime(df['DateTime'])
  df=df[colls]
  df['Date']= df['DateTime']
  df = df.drop('DateTime', axis=1, inplace=False)
  df['Date'] = df['Date'].astype(str)
  if 'security' in ftype:
      df['TransID'] = df['TransID'].astype(str)
  return df

gdrivepath = "/content/drive/MyDrive/Colab Notebooks/My NN stuff/alteon/dbe.security.db"

df_aw = aw_to_pd(gdrivepath)



**Setting up langchain components and pipeline**

In [ ]:
python_repl_appwall = PythonREPL()
python_repl_appwall.globals['df'] = df_aw


# Permanently changes the pandas settings
python_repl_appwall.run(pd.set_option('display.max_rows', None))
python_repl_appwall.run(pd.set_option('display.max_columns', None))
python_repl_appwall.run(pd.set_option('display.width', None))
python_repl_appwall.run(pd.set_option('display.max_colwidth', 500))


name_appwall = "python_repl_for_pandas_dataframes_appwall_database"

description_appwall_2= """
A Python shell to execute pandas commands on appwall dataframe 'df' which contains appwall database (db).
The column names in this pandas dataframe df can be found in this list = ['Date','TargetPort','TargetIP','TransID','TunnelID','IsPassiveMode','Title','URI','Description']
The dataframe df contains entries in increasing order of time,
such that the latest entry is at the end and earliest entry is at the beginning.
The final answer should always be enclosed in a print() function
"""

repl_tool_appwall = Tool(
    name=name_appwall,
    description=description_appwall_2,
    func=python_repl_appwall.run

)


appwall_llm = llmMist.bind_tools([repl_tool_appwall])

sys_appwall= """
Only if user query is talking about 'appwall' or 'WAF' or 'AW' or 'CWAF' or 'database' or 'db',
do they have questions about the data stored in the pandas dataframe df for appwall database.
"""

prompt_appwall = ChatPromptTemplate.from_messages([
        ("system", sys_appwall),
        ("human", "{query}"),
        ])

setupinputs = {"query": RunnablePassthrough()}


def check_print(result: str) -> str:
  if 'print' in result:
    return result
  else:
    return f"print({result})"

def out_parser_appwall(result: str) -> str:

  """Parses the output of python repl and returns the final formatted output"""

  no_answer = "I couldnt locate what you are asking for in the appwall db."
  attributeerr = "I dont have an answer to your query. Please try to revise your question and ask again. "
  if not result:
    return no_answer
  if 'Empty DataFrame' in result:
    return no_answer
  if 'AttributeError' in result:
    return attributeerr
  return result

def toolingfunc_appwall(x: str) -> str:
  try:
    return x.tool_calls[0]["args"]['__arg1']
  except:
    # No tool/arg found. This need to be different message encouraging user to rephrase the question
    return "print('I dont have an answer to your query. Kindly try rephrasing your question or adding more specificity to it.')"


def notebook_rate_print(data: str) -> str:
  if not data:
    return data
  data_size = len(data)
  if data_size > 999000:
    return "Response data size exceeds display limit and will only be partially displayed. Please refine your query and ask again!\n\n" +data[:999000]
  else:
    return data

appwall_chain = setupinputs| prompt_appwall | appwall_llm | RunnableLambda(toolingfunc_appwall) | RunnableLambda(check_print) | python_repl_appwall.run | RunnableLambda(notebook_rate_print) | RunnableLambda(out_parser_appwall)






**Call chain with your questions**

In [ ]:
print ("query: ", query)
resp = appwall_chain.invoke({'query':query})
print(resp)
